In [27]:
#!pip3 install pymysql

In [28]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql

### Opiods are involved in the majority of drug over dose cases. in 2015 there were 33,091 overdose deaths in the US. The number of overdoses has quadrupled since 1991.
#### In 2015, the five states with the highest rates of death due to drug overdose were West Virginia (41.5 per 100,000), New Hampshire (34.3 per 100,000), Kentucky (29.9 per 100,000), Ohio (29.9 per 100,000), and Rhode Island (28.2 per 100,000)
Source: https://data.world/cdc/2015-drug-overdose-deaths

# Load Data

In [29]:
# Data from: https://data.world/cdc/2015-drug-overdose-deaths
# Total deaths and death rate per 100,000
deaths_2015 = pd.read_csv('./resources/2015-death-data.csv')
deaths_2015.head(3)

,State,Range,Rate,Number,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,AL,13.6 to 16.0,15.7,736,NaN,NaN,NaN,NaN
1,AK,13.6 to 16.0,16.0,122,NaN,NaN,NaN,NaN
2,AZ,18.6 to 21.0,19.0,"1,274",NaN,NaN,NaN,NaN


In [30]:
#CDC data on Morphine Milligram equivalents by county. Based on raw data from 
#QuintilesIMS Transactional Data Warehouse
#https://data.world/jmpichar/opioid-prescriptions/workspace/file?agentid=associatedpress&datasetid=opioid-prescriptions-2010-2015&filename=Opioid_prescription_amounts.csv
#
opioids_2010_2015_rx = pd.read_csv('./resources/Opioid_prescription_amounts.csv')
opioids_2010_2015_rx.head(3)

,FIPS,State,County,MME_PerCap_2010,MME_PerCap_2015,Quartile_2015,2010-2015_Change
0,2050,AK,BETHEL,NaN,NaN,NaN,NaN
1,2068,AK,DENALI,NaN,NaN,NaN,NaN
2,2013,AK,ALEUTIANS EAST,NaN,NaN,NaN,NaN


In [31]:
# mulltiple cause of death data on CDC Wonder
# The crude rate is per 100,000.
# The population figures are briged-race estimates. The exceptions being years 2000 and 2010, in which Census counts are used.
# v1.1: Added Opioid Prescriptions Dispensed by US Retailers in that year (millions).
#https://data.world/health/opioid-overdose-deaths
mcd_1999_2014 = pd.read_csv('./resources/multiple_cause_death_1999-2014.csv')
mcd_1999_2014.head(3)

,State,Year,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Prescriptions Dispensed by US Retailers in that year (millions)
0,Alabama,1999,39,4430141,0.9,0.6,1.2,116
1,Alabama,2000,46,4447100,1,0.8,1.4,126
2,Alabama,2001,67,4467634,1.5,1.2,1.9,138


In [32]:
# load states facts csv and create a new data frame mapping state names to 
# state abbreviation used to trnasform the data in other tables
states_facts = pd.read_csv('./resources/states_facts.csv', index_col='Sr.No')
state_abbrevs = states_facts[['State', 'Abbr.']].set_index('State')
state_abbrevs.head(3)

,Abbr.
State,
Alabama,AL
Alaska,AK
Arizona,AZ


# Transformation

## 2015-death-data
### Data is clean just Drop unused columns
### Rename columns

In [33]:
deaths_2015.columns

Index(['State', 'Range', 'Rate', 'Number', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7'],
      dtype='object')

In [34]:
deaths_2015 = deaths_2015[['State', 'Rate', 'Number']]

In [35]:
deaths_2015.rename(columns= {'Rate':'2015 Rate','Number':'2015 Deaths'}, inplace=True)

In [36]:
deaths_2015.head(3)

,State,2015 Rate,2015 Deaths
0,AL,15.7,736
1,AK,16.0,122
2,AZ,19.0,"1,274"


## Opioid_prescription_amounts
### Drop Quartile_2015
### Rename Columns
#### MME_PerCap_2010 -> 2010
#### MME_PerCap_2015 -> 2015
#### 2010-2015_Change -> Percent Change
#### Drop NAN  if data is missing for both years
    

In [37]:
opioids_2010_2015_rx.columns

Index(['FIPS', 'State', 'County', 'MME_PerCap_2010', 'MME_PerCap_2015',
       'Quartile_2015', '2010-2015_Change'],
      dtype='object')

In [38]:
# Drop 'Quartile_2015'
opioids_2010_2015_rx = opioids_2010_2015_rx[['FIPS', 'State', 'County', 'MME_PerCap_2010', 'MME_PerCap_2015', '2010-2015_Change']]

In [39]:
# Rename columns
opioids_2010_2015_rx.columns = ['FIPS', 'State', 'County', '2010', '2015', '2010-2015_Change']
#opioids_2010_2015_rx.head(3)

In [40]:
# Drop NaN
opioids_2010_2015_rx.dropna(subset =['2010', '2015'], inplace=True)

opioids_2010_2015_rx.set_index(np.arange(len(opioids_2010_2015_rx.index)),inplace=True)
opioids_2010_2015_rx.head(3)

,FIPS,State,County,2010,2015,2010-2015_Change
0,2122,AK,KENAI PENINSULA,1362.1,1343.7,Stable
1,2170,AK,MATANUSKA SUSITNA,930.4,869.3,Stable
2,2020,AK,ANCHORAGE,863.8,808.9,Stable


## mulltiple cause of death data
### Create a new DataFrame with death rates for 2010 which is missing in other death rate data
### Drop unecessary columns
### Rename State names with Abreviation
### Rename remaining columns to match other data sets



In [41]:
#Create a new DataFrame with death rates for 2010
mask = mcd_1999_2014['Year'] == 2010
deaths_2010 = mcd_1999_2014[mask]
deaths_2010.set_index(np.arange(len(deaths_2010.index)), inplace=True)
deaths_2010.head()

,State,Year,Deaths,Population,Crude Rate,Crude Rate Lower 95% Confidence Interval,Crude Rate Upper 95% Confidence Interval,Prescriptions Dispensed by US Retailers in that year (millions)
0,Alabama,2010,194,4779736,4.1,3.5,4.6,210
1,Alaska,2010,71,710231,10,7.8,12.6,210
2,Arizona,2010,650,6392017,10.2,9.4,11,210
3,Arkansas,2010,198,2915918,6.8,5.8,7.7,210
4,California,2010,2059,37253956,5.5,5.3,5.8,210


In [42]:
deaths_2010.columns

Index(['State', 'Year', 'Deaths', 'Population', 'Crude Rate',
       'Crude Rate Lower 95% Confidence Interval',
       'Crude Rate Upper 95% Confidence Interval',
       'Prescriptions Dispensed by US Retailers in that year (millions)'],
      dtype='object')

In [43]:
#deaths_2010.rename(columns = {'Prescriptions Dispensed by US Retailers in that year (millions)':'Prescriptions Dispensed (millions)'},inplace=True)

In [44]:
# drop un used columns : 'Year', Population', 'Crude Rate', 'Crude Rate Lower 95% Confidence Interval',
#'Crude Rate Upper 95% Confidence Interval','Prescriptions Dispensed by US Retailers in that year (millions)'
deaths_2010 = deaths_2010[['State', 'Deaths']]

In [45]:
deaths_2010.rename(columns = {'Deaths':'2010 Deaths'}, inplace=True)
deaths_2010.head(3)

,State,2010 Deaths
0,Alabama,194
1,Alaska,71
2,Arizona,650


In [46]:
state_abbrevs.loc['Alabama'][0]

'AL'

In [47]:
## Convert State names to state Abbreviations
for index,row in deaths_2010.iterrows():
    state = row['State']
    #print(state)
    abrev = state_abbrevs.loc[state][0]
    deaths_2010.loc[index, 'State'] = abrev

In [48]:
deaths_2010.head(3)

,State,2010 Deaths
0,AL,194
1,AK,71
2,AZ,650


In [49]:
deaths_2015.head(3)

,State,2015 Rate,2015 Deaths
0,AL,15.7,736
1,AK,16.0,122
2,AZ,19.0,"1,274"


In [50]:
deafs_2010_2015 = deaths_2010.merge(deaths_2015, how = "inner", on = "State")
deafs_2010_2015.head(3)

,State,2010 Deaths,2015 Rate,2015 Deaths
0,AL,194,15.7,736
1,AK,71,16.0,122
2,AZ,650,19.0,"1,274"


# Load data into MySQL Database

## Create database connection

In [52]:
connection_string = "root:password@127.0.0.1:3306/opioids_db"
engine = create_engine(f'mysql+pymysql://{connection_string}')

## Load DataFrames into database

In [54]:
deafs_2010_2015.to_sql(name='deaths', con=engine, if_exists='append', index=True)

In [55]:
opioids_2010_2015_rx.to_sql(name='prescriptions', con=engine, if_exists='append', index=True)